## Question 2

In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import Model, GRB, quicksum

In [3]:
gym_data = pd.read_csv("/Users/shivamverma/Downloads/updated_gym_data.csv")
gym_data.head()

,Exercise,Category,BodyPart,Equipment,Difficulty,Stimulus-to-Fatigue,Expected Time,Hypertrophy Rating
0,Bench Press With Short Bands,Powerlifting,Chest,Bands,Beginner,0.817884,15.518089,0.596124
1,Hip Lift with Band,Powerlifting,Glutes,Bands,Beginner,0.768902,14.655351,0.623237
2,Band Good Morning (Pull Through),Powerlifting,Hamstrings,Bands,Beginner,0.792188,16.292358,0.601159
3,Speed Box Squat,Powerlifting,Quadriceps,Bands,Intermediate,0.599044,17.109781,0.800347
4,Partner plank band row,Strength,Abdominals,Bands,Intermediate,0.730726,14.212727,0.461565


In [32]:
gym_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Exercise             2637 non-null   object 
 1   Category             2637 non-null   object 
 2   BodyPart             2637 non-null   object 
 3   Equipment            2630 non-null   object 
 4   Difficulty           2637 non-null   object 
 5   Stimulus-to-Fatigue  2637 non-null   float64
 6   Expected Time        2637 non-null   float64
 7   Hypertrophy Rating   2637 non-null   float64
dtypes: float64(3), object(5)
memory usage: 164.9+ KB


### (a) How many decision variables are in the optimization problem and what is their range?

    - There are 2637 decision variables in the optimization problem, because 2,637 exercises in the dataset.
    - The range of each exercise is between 0-1. If an exercise accounts for 4% of the total, it should be written as 0.04.

In [4]:
# Extract relevant columns for the model
exercises = gym_data['Exercise'].tolist()
hypertrophy_ratings = gym_data['Hypertrophy Rating'].tolist()
stimulus_to_fatigue = gym_data['Stimulus-to-Fatigue'].tolist()
body_parts = gym_data['BodyPart'].tolist()
equipment = gym_data['Equipment'].tolist()
difficulty = gym_data['Difficulty'].tolist()


In [5]:
# Initialize the model by Gurobi
model = gp.Model("Workout Program Optimization")

Set parameter Username
Set parameter LicenseID to value 2615603
Academic license - for non-commercial use only - expires 2026-01-27


In [7]:

# Decision variables: proportion of workout for each exercise
x = model.addVars(len(exercises), lb=0, ub=1, name="Exercise_Proportion")

# Objective: Maximize the total hypertrophy rating
model.setObjective(sum(hypertrophy_ratings[i] * x[i] for i in range(len(exercises))), GRB.MAXIMIZE)


### Adding Constraints

In [31]:
# Constraints
# 1. The sum of all proportions must equal 1 (100% of the workout program)
model.addConstr(sum(x[i] for i in range(len(exercises))) == 1, "Total_Proportion")



# 2. No single exercise can account for more than 5% of the total workout program
for i in range(len(exercises)):
    model.addConstr(
        x[i] <= 0.05, name=f"MaxSingleExercise_{i}"
    )

# 3. Each body part should be included in at least 2.5% of the program, except for Traps, Neck, and Forearms, which require at least 0.5%
#   Abdominals, which should make up at least 4%
body_part_groups = gym_data.groupby('BodyPart').groups
min_allocations = {
    "Traps": 0.005, "Neck": 0.005, "Forearms": 0.005, "Abdominals": 0.04
}
for body_part, indices in body_part_groups.items():
    min_allocation = min_allocations.get(body_part, 0.025)
    model.addConstr(
        gp.quicksum(x[i] for i in indices) >= min_allocation,
        name=f"MinAllocation_{body_part}"
    )

# 4. Leg muscles (i.e., Adductors, Abductors, Calves, Glutes, Hamstrings, Quadriceps) must receive at least 2.6 times the allocation than all upper body exercises combined.
leg_muscle_categories = [body_part for body_part in body_part_groups.keys() if "Leg" in body_part or body_part in ["Adductors", "Abductors", "Calves", "Glutes", "Hamstrings", "Quadriceps"]]
upper_body_muscles = [m for m in body_part_groups.keys() if m not in leg_muscle_categories]
model.addConstr(
    gp.quicksum(x[i] for i in gym_data.index if gym_data['BodyPart'].iloc[i] in leg_muscle_categories) >=
    2.6 * gp.quicksum(x[i] for i in gym_data.index if gym_data['BodyPart'].iloc[i] in upper_body_muscles),
    name="LegMusclesAllocation"
)

# 5. The proportion of the workout program devoted to Biceps and Triceps training should be equal, as should the proportion of the program devoted to Chest and All Back exercises combined.
# Proportion of Biceps = Triceps = Chest + All Back exercises
biceps_indices = gym_data[gym_data["BodyPart"] == "Biceps"].index
triceps_indices = gym_data[gym_data["BodyPart"] == "Triceps"].index
chest_indices = gym_data[gym_data["BodyPart"] == "Chest"].index
all_back_indices = gym_data[gym_data["BodyPart"] == "All Back"].index
model.addConstr(
    gp.quicksum(x[i] for i in biceps_indices) ==
    gp.quicksum(x[i] for i in triceps_indices),
    name="BicepsEqualsTriceps"
)


# 6. The overall stimulus-to-fatigue (SFR) ratio of the program should not exceed 0.55.
model.addConstr(
    gp.quicksum(
        stimulus_to_fatigue[i] * x[i] for i in range(len(exercises))
    ) <= 0.55,
    name="SFRConstraint"
)


# 7. The proportion of Beginner exercises should be at least 1.4× that of Intermediate exercises, and the proportion of Intermediate exercises should be at least 1.1× that of Advanced exercises.
beginner_indices = gym_data[gym_data['Difficulty'] == "Beginner"].index
intermediate_indices = gym_data[gym_data['Difficulty'] == "Intermediate"].index
advanced_indices = gym_data[gym_data['Difficulty'] == "Advanced"].index
model.addConstr(
    gp.quicksum(x[i] for i in beginner_indices) >=
    1.4 * gp.quicksum(x[i] for i in intermediate_indices),
    name="BeginnerProportion"
)
model.addConstr(
    gp.quicksum(x[i] for i in intermediate_indices) >=
    1.1 * gp.quicksum(x[i] for i in advanced_indices),
    name="IntermediateProportion"
)




# 8. Strongman exercises should consist of less than 8% of the workout program, Powerlifting exercises should exceed 9%, and Olympic Weightlifting exercises should exceed 10%.
strongman_indices = gym_data[gym_data['Category'] == "Strongman"].index
powerlifting_indices = gym_data[gym_data['Category'] == "Powerlifting"].index
olympic_indices = gym_data[gym_data['Category'] == "Olympic Weightlifting"].index
model.addConstr(
    gp.quicksum(x[i] for i in strongman_indices) <= 0.08,
    name="StrongmanProportion"
)
model.addConstr(
    gp.quicksum(x[i] for i in powerlifting_indices) >= 0.09,
    name="PowerliftingProportion"
)
model.addConstr(
    gp.quicksum(x[i] for i in olympic_indices) >= 0.10,
    name="OlympicProportion"
)

# 9. The proportion of the workout program utilizing Barbells, Dumbbells, Machines, Cables, the E-Z Curl bar, and Bands should exceed 60% of the workout program.
equipment = gym_data['Equipment']
model.addConstr(
    gp.quicksum(x[i] for i in range(len(exercises)) if equipment[i] in ['Barbells', 'Dumbbells', 'Machines', 'Cables', 'E-Z Curl bar', 'Bands']) >= 0.6,
    name="EquipmentMin"
)

# Optimize the model
model.optimize()


# Display results
if model.status == gp.GRB.OPTIMAL:
    print(f"Optimal Hypertrophy Rating: {model.objVal:.4f}")
    print("\nSelected Exercises and Allocations:")
    for i, v in enumerate(x.values()):
        if v.x > 0:
            print(f"{exercises[i]}: {v.x:.4f}")

else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 45081 rows, 5274 columns and 203683 nonzeros
Model fingerprint: 0x5fb6908b
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 45055 rows and 2637 columns
Presolve time: 0.05s
Presolved: 26 rows, 2637 columns, 13677 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier performed 0 iterations in 0.05 seconds (0.05 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex

Solved in 8 iterations and 0.06 seconds (0.05 work units)
Infeasible model
No optimal solution found.


### (b) The objective is to ”allocate a proportion of the workout program to each exercise.” Explain why this approach is more practical than specifying exact exercises for each session.

#### Primarily fulfils personalized needs
1. Needs for adjustment: Everybody has unique needs, skills, and objectives, and they can better modify the ratios based on their own circumstances.
2. Flexibility: You don't have to change the plan if you add additional workouts; you can simply change the proportions. 

### (c) Using Gurobi, what is the optimal hypertrophy rating using all constraints?


In [33]:
if model.status == GRB.OPTIMAL:
    optimal_rating = model.objVal
    print(f"Optimal Hypertrophy Rating: {optimal_rating}")
else:
    print("No optimal solution found.")


No optimal solution found.


In [35]:
# Initialize the Gurobi model
model = gp.Model("Workout Program Optimization")

# Create decision variables for each exercise
x = model.addVars(
    len(x), lb=0, ub=1, name="Exercise"
)

# Objective: Maximize overall hypertrophy rating
model.setObjective(
    gp.quicksum(
        hypertrophy_ratings[i] * x[i] for i in range(len(exercises))
    ),
    gp.GRB.MAXIMIZE
)

In [36]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2637 columns and 0 nonzeros
Model fingerprint: 0xd8446ac0
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 2637 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4720513e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.472051303e+03


In [38]:
# Display results
if model.status == gp.GRB.OPTIMAL:
    print(f"Optimal Hypertrophy Rating: {model.objVal:.4f}")
    print("\nSelected Exercises and Allocations:")
    for i, v in enumerate(x.values()):
        if v.x > 0:
            print(f"{x[i]}: {v.x:.4f}")

else:
    print("No optimal solution found.")

Optimal Hypertrophy Rating: 1472.0513

Selected Exercises and Allocations:
<gurobi.Var Exercise[0] (value 1.0)>: 1.0000
<gurobi.Var Exercise[1] (value 1.0)>: 1.0000
<gurobi.Var Exercise[2] (value 1.0)>: 1.0000
<gurobi.Var Exercise[3] (value 1.0)>: 1.0000
<gurobi.Var Exercise[4] (value 1.0)>: 1.0000
<gurobi.Var Exercise[5] (value 1.0)>: 1.0000
<gurobi.Var Exercise[6] (value 1.0)>: 1.0000
<gurobi.Var Exercise[7] (value 1.0)>: 1.0000
<gurobi.Var Exercise[8] (value 1.0)>: 1.0000
<gurobi.Var Exercise[9] (value 1.0)>: 1.0000
<gurobi.Var Exercise[10] (value 1.0)>: 1.0000
<gurobi.Var Exercise[11] (value 1.0)>: 1.0000
<gurobi.Var Exercise[12] (value 1.0)>: 1.0000
<gurobi.Var Exercise[13] (value 1.0)>: 1.0000
<gurobi.Var Exercise[14] (value 1.0)>: 1.0000
<gurobi.Var Exercise[15] (value 1.0)>: 1.0000
<gurobi.Var Exercise[16] (value 1.0)>: 1.0000
<gurobi.Var Exercise[17] (value 1.0)>: 1.0000
<gurobi.Var Exercise[18] (value 1.0)>: 1.0000
<gurobi.Var Exercise[19] (value 1.0)>: 1.0000
<gurobi.Var Exe

In [39]:
if model.status == GRB.OPTIMAL:
    optimal_rating = model.objVal
    print(f"Optimal Hypertrophy Rating: {optimal_rating}")
else:
    print("No optimal solution found.")

Optimal Hypertrophy Rating: 1472.0513034809994
